In [65]:
import pandas as pd

# Load the clean dataset with explicit encoding
df = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1')
df_head = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1', nrows=1)

# Get the first row as column names
column_names=df_head.columns.tolist()

# Drop the first two rows (header and unit information)
df = df[2:]

# Set the column names
df.columns = column_names

# Iterate through each column starting from the second column (index 1)
response_tally = []

for column in df.columns[1:]:
    responses = df[column].value_counts().reset_index()
    responses.columns = ['Response', 'Count']
    responses['Column'] = column
    response_tally.append(responses)

# Concatenate the response tally dataframes
result_df = pd.concat(response_tally, ignore_index=True)

# Save the result to an Excel file
result_df.to_excel('numberOfResponsesPerClass.xlsx', index=False)


/tmp/ipykernel_4803/3138198795.py:4: DtypeWarning: Columns (0,15,43,57,73,88,104,118,126,132,170,200,215,248,272,281,294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1')


In [66]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel('response_tally.xlsx')

# Initialize an empty dictionary to hold string tallies
column_tally = {}

# Loop through each row in the "Column" column
for item in df['Column']:
    if item in column_tally:
        column_tally[item] += 1
    else:
        column_tally[item] = 1

# Convert the column tally dictionary to a DataFrame
output_df = pd.DataFrame(list(column_tally.items()), columns=['Column', 'Tally'])

# Save the DataFrame to a new Excel file
output_df.to_excel('numberOfclassesPerQuestion.xlsx', index=False)
